
<img src="https://static.independent.co.uk/s3fs-public/thumbnails/image/2012/07/14/21/SU18-slug.jpg" width="20%">

In [1]:
import pandas as pd
import numpy as np
import requests
import time

from bs4 import BeautifulSoup


In [2]:
# required score
score_req = 1000
# desired post count
post_count = 10_000

# retriving starting data
fifth_url = f"https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E{score_req}"
first_url = f"https://api.pushshift.io/reddit/search/submission?subreddit=firstworldproblems&size=500&score%3E{score_req}"

fifth_res = requests.get(fifth_url)
first_res = requests.get(first_url)


print(fifth_res.status_code, first_res.status_code)


fifth_res = fifth_res.json()
first_res = first_res.json()

200 200


In [3]:
# checking data structure

fifth_res_df = pd.DataFrame(fifth_res['data'])
fifth_res_df[['title', 'selftext', 'created_utc', 'permalink']].sort_values('created_utc').head(1)

,title,selftext,created_utc,permalink
499,I'd been outside of my body for several weeks ...,,1577142727,/r/fifthworldproblems/comments/ees9ue/id_been_...


In [4]:
first_res_df = pd.DataFrame(first_res['data'])
first_res_df[['title', 'selftext', 'created_utc', 'permalink']].sort_values('created_utc').head(1)

,title,selftext,created_utc,permalink
499,Poor Mylow how will he ever cope,,1578565489,/r/firstworldproblems/comments/em7th2/poor_myl...


In [5]:
# setup for urls to be added to
fifth_urls = [fifth_url]
first_urls = [first_url]
# finding the earliest post date to prevent post overlap
fifth_date = fifth_res_df.sort_values('created_utc').reset_index()['created_utc'][0]
first_date = first_res_df.sort_values('created_utc').reset_index()['created_utc'][0]

# looping for all urls
for i in range(int(post_count / 500)):
    # calling api
    fifth_temp = requests.get(f"https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E{score_req}&before={fifth_date}")
    first_temp = requests.get(f"https://api.pushshift.io/reddit/search/submission?subreddit=firstworldproblems&size=500&score%3E{score_req}&before={first_date}")
    
    fifth_temp = fifth_temp.json()
    first_temp = first_temp.json()
    #retriving earliest post date
    fifth_temp = pd.DataFrame(fifth_temp['data'])
    first_temp = pd.DataFrame(first_temp['data'])
    #adding api calls with earliest post date to list
    fifth_urls.append(f"https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E{score_req}&before={fifth_date}")
    first_urls.append(f"https://api.pushshift.io/reddit/search/submission?subreddit=firstworldproblems&size=500&score%3E{score_req}&before={first_date}")
    
    
    #setting the new earliest post date
    fifth_date = fifth_temp.sort_values('created_utc').reset_index()['created_utc'][0]
    first_date = first_temp.sort_values('created_utc').reset_index()['created_utc'][0]
    #resting api
    time.sleep(1)


In [6]:
#double checking urls
fifth_urls, first_urls

(['https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E1000',
  'https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E1000&before=1577142727',
  'https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E1000&before=1574210758',
  'https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E1000&before=1571181090',
  'https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E1000&before=1568809640',
  'https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E1000&before=1565588987',
  'https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E1000&before=1562189665',
  'https://api.pushshift.io/reddit/search/submission?subreddit=fifthworldproblems&size=500&score%3E1000&before=1559539008',
  'https://api.pushshift.i

In [7]:
# setting up list for dictionaries
results = []
# looping through each api call
for url in (fifth_urls + first_urls):
    #calling try to prevent failed calls ending code
    try:
        #grabbing data from api call
        res = requests.get(url)
        res = res.json()
        #looping through api call data
        for val in res['data']:
            
            #initializing a dict for desired data values
            temp_dict = {}
            #setting key pairs
            temp_dict['title'] = val['title']
            temp_dict['selftext'] = val['selftext']
            temp_dict['link'] = val['permalink']
            temp_dict['created_utc'] = val['created_utc']
            #running simple if else block to make a key encoding subreddit source
            if url in fifth_urls:
                temp_dict['is_fifth'] = 1
            else:
                temp_dict['is_fifth'] = 0
            
            # adding dictionary of desired data to results
            results.append(temp_dict)
    except:
        print('api call failed')
    #resting api
    time.sleep(3)



api call failed
api call failed


In [8]:
# creating a pandas dataframe from results
results = pd.DataFrame(results)

In [9]:
# checking for an odd repeated value
[val[1][0] for val in results.iterrows() if val[1][1] == 'https://youtu.be/tmmCW8Cz7mE']

['YouTube won’t let me add a mashup of “Thomas the Tank Engine” and “My Neck, My Back” to a playlist because it thinks the video is for kids',
 'YouTube won’t let me add a mashup of Thomas the Tank Engine and My Neck, My Back because it thinks it’s for kids']

In [10]:
# looking at commonly repeated selftext
results['selftext'].value_counts()

                                                                                                                                                                                                                                                                                                                                                                                               12486
[removed]                                                                                                                                                                                                                                                                                                                                                                                        488
[deleted]                                                                                                                                                                                                                     

In [11]:
# exporting results
results.to_csv('data/results_dirty.csv', index=False)